In [15]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine

In [16]:
pd.options.plotting.backend = "plotly"

In [17]:
# Redshift connection
f = open('/Users/gabrielreynoso/Documents/Queries/db_klarprod_connection.txt', 'r')
postgres_str = f.read()
f.close()
cnx = create_engine(postgres_str)

In [18]:
query = '''
select *
from unit_economics.debit_monthly_economics
'''

In [19]:
debit_df_whole = pd.read_sql_query(query, cnx)

In [20]:
#censoring
debit_df = debit_df_whole[debit_df_whole['debit_business_month'] >= 0]
debit_df = debit_df[debit_df['transaction_active_customers'] > 1]
debit_df['actual_month'] = debit_df.apply(lambda x: x.debit_monthly_cohort + pd.DateOffset(months=x.debit_business_month, days=0), axis=1)

In [21]:
profit_loss = debit_df.groupby(by='actual_month').aggregate({
    'contribution_profit_30': 'sum',
    'gross_interchange_revenue':'sum',
    'gross_float_revenue':'sum',
    'gross_interest_revenue': 'sum',
    'gross_paid_late_fee_revenue_30': 'sum',
    'gross_revenue_30': 'sum',
    'gross_npl_30': 'sum'}).reset_index()
profit_loss_styled = profit_loss.style.format(formatter={'revenues_usd': '{:,.2f}', 'losses_usd': '{:,.2f}'})

In [22]:
profit_loss[profit_loss['actual_month']>'2021-03'].to_clipboard(index=False)

In [23]:
profit_loss

,actual_month,contribution_profit_30,gross_interchange_revenue,gross_float_revenue,gross_interest_revenue,gross_paid_late_fee_revenue_30,gross_revenue_30,gross_npl_30
0,2020-07-01,5.592463e+04,7.577560e+04,0.000000,2.489350e+05,9.930250e+03,3.346409e+05,2.787162e+05
1,2020-08-01,2.082549e+04,2.074044e+05,9382.953730,1.846350e+05,9.492100e+03,4.109144e+05,3.900889e+05
2,2020-09-01,-2.732623e+05,2.735838e+05,50797.347005,3.331700e+05,1.692757e+04,6.744787e+05,9.477410e+05
3,2020-10-01,3.235779e+05,4.166630e+05,65565.474118,2.986750e+05,5.540000e+03,7.864435e+05,4.628656e+05
4,2020-11-01,6.717740e+05,6.019463e+05,86707.174499,2.202740e+05,3.950000e+03,9.128775e+05,2.411035e+05
5,2020-12-01,-2.862798e+05,7.336917e+05,124149.244725,8.198750e+05,9.753000e+03,1.687469e+06,1.973749e+06
6,2021-01-01,-8.135725e+05,8.376813e+05,141274.230414,1.046425e+06,1.483000e+04,2.040211e+06,2.853783e+06
7,2021-02-01,-1.619331e+06,1.047855e+06,177283.055316,2.079175e+06,3.144400e+04,3.335757e+06,4.955089e+06
8,2021-03-01,-5.857726e+06,1.175130e+06,216960.904918,5.068587e+06,1.185281e+05,6.579206e+06,1.243693e+07
9,2021-04-01,-7.538903e+06,1.405453e+06,247115.029190,6.679250e+06,1.397650e+05,8.471583e+06,1.601049e+07


# With Censoring

In [24]:
#full-censoring
debit_df = debit_df[debit_df['full_loan_censoring_30'] == 1]

In [25]:
profit_loss = debit_df.groupby(by='actual_month').aggregate({
    'contribution_profit_30': 'sum',
    'gross_interchange_revenue':'sum',
    'gross_float_revenue':'sum',
    'gross_interest_revenue': 'sum',
    'gross_paid_late_fee_revenue_30': 'sum',
    'gross_revenue_30': 'sum',
    'gross_npl_30': 'sum'}).reset_index()
profit_loss_styled = profit_loss.style.format(formatter={'revenues_usd': '{:,.2f}', 'losses_usd': '{:,.2f}'})

In [26]:
profit_loss_styled

,actual_month,contribution_profit_30,gross_interchange_revenue,gross_float_revenue,gross_interest_revenue,gross_paid_late_fee_revenue_30,gross_revenue_30,gross_npl_30
0,2020-07-01 00:00:00,55924.631036,75775.601036,0.000000,248935.000000,9930.250000,334640.851036,278716.220000
1,2020-08-01 00:00:00,20825.485232,207404.371502,9382.953730,184635.000000,9492.100000,410914.425232,390088.940000
2,2020-09-01 00:00:00,-273262.321367,273583.781628,50797.347005,333170.000000,16927.570000,674478.698633,947741.020000
3,2020-10-01 00:00:00,323577.934504,416663.040386,65565.474118,298675.000000,5540.000000,786443.514504,462865.580000
4,2020-11-01 00:00:00,671773.984057,601946.279558,86707.174499,220274.000000,3950.000000,912877.454057,241103.470000
5,2020-12-01 00:00:00,-286279.797017,733691.708258,124149.244725,819875.000000,9753.000000,1687468.952983,1973748.750000
6,2021-01-01 00:00:00,-813572.480998,837681.318588,141274.230414,1046425.000000,14830.000000,2040210.549002,2853783.030000
7,2021-02-01 00:00:00,-1619331.159992,1047855.344692,177283.055316,2079175.000000,31444.000000,3335757.400008,4955088.560000
8,2021-03-01 00:00:00,-5857725.873816,1175130.161266,216960.904918,5068587.000000,118528.110000,6579206.176184,12436932.050000
9,2021-04-01 00:00:00,-7538903.045066,1405452.715744,247115.029190,6679250.000000,139765.000000,8471582.744934,16010485.790000


In [27]:
profit_loss[profit_loss['actual_month']>'2021-03'].to_clipboard(index=False)